# 17. SAC（Soft Actor-Critic）：“平滑“ Actor-Critic
> 尽管 **DDPG** 属于 **离线策略算法** ，具有一定的样本效率。但是它的训练不稳定，收敛性较差，需要细致的超参数调整，难以适应复杂环境。
> 2018 年，Tuomas 等人提出了一个更加稳定的 **离线策略算法**： [Soft Actor-Critic: Off-Policy Maximum Entropy Deep Reinforcement Learning with a Stochastic Actor](https://arxiv.org/abs/1801.01290)
> **SAC** 的前身是 **Soft Q-learning**，它们都属于 **最大熵** 强化学习的范畴。**Soft Q-learning** 不存在一个 **显式的策略函数**，而是使用一个 **Q函数的最大熵（Boltzmann）分布**：
$$\pi(a|s)=\frac{e^{\frac{Q(s,a)}{\alpha}}}{\sum_{a^{\prime}}e^{\frac{Q(s,a^{\prime})}{\alpha}}}$$
- 当$\alpha$较大时,$\frac{Q(s,a)}{\alpha}$的值变小,导致所有的$\exp$值接近 1，所有动作的概率大致相等，策略变得更为均匀，鼓励 **更多的探索**
- 当$\alpha$较小时,$\frac{Q(s,a)}{\alpha}$的值变大,导致所有的$\exp$值远大于其他动作，所有动作的概率差异变大，倾向于 **更多的利用**

> 目前，在**无模型**的强化学习算法中，**SAC** 是一个非常高效的算法，它学习一个 **随机性策略**，在不少标准环境中取得了领先的成绩。

## 17.1 SAC算法原理
- 最大熵强化学习
- Soft 策略迭代
- SAC 网络结构

### 最大熵强化学习
> **熵（Entropy）** 本质上是**不确定性**或**信息量的期望**的度量。它最早由 **香农在信息论** 中提出，用来描述一个随机变量的 **不确定程度**
> 如果一个离散随机变量 $X$ 的概率分布是：
$$
P(X = x_i) = p_i,\quad i=1,\dots, n
$$
> 那么它的**信息熵**定义为：
$$
H(X) = - \sum_{i=1}^n p_i \log p_i
$$

- $p_i$ 越均匀，熵越大，表示越不确定
- $p_i$ 越集中（某个事件几乎必然发生），熵越小，甚至为 $0$（完全确定）
> **例子**：
- 公平硬币：$p(\text{正})=p(\text{反})=0.5$，熵最大 $H=1$ （最不确定）
- 偏置硬币：$p(\text{正})=0.9$，熵小于 $1$（差不多确定）
- 完全确定事件：$p=1$ 或 $p=0$，熵为 $0$（无不确定性）

> 如果是连续随机变量 $X$，熵的定义为**微分熵**：
$$
H(X) = -\int p(x) \log p(x) \, dx
$$
---
> 传统强化学习的目标是在一条轨迹$\tau$下 **最大化期望累积回报**：
$$
J(\pi) = \mathbb{E}_{\tau\sim\pi} \left[ \sum_{t=0}^{\infty} \gamma^t r(s_t,a_t) \right]
$$
- 目标**只关心奖励**，并不会**刻意**鼓励探索，可能会早早陷入次优策略
---
> 而 **最大熵强化学习** 把策略的 **不确定性（熵）** 也纳入优化目标：
$$
J(\pi) = \mathbb{E}_{\tau\sim\pi} \left[ \sum_{t=0}^{\infty} \gamma^t \big( r(s_t,a_t) + \alpha \, \mathcal{H}(\pi(\cdot|s_t)) \big) \right]
$$

- $\mathcal{H}(\pi(\cdot|s_t)) = - \sum_a \pi(a|s_t) \log \pi(a|s_t)$ 表示在状态 $s_t$ 下动作分布的熵
- $\alpha$ 是**温度参数**，权衡**奖励最大化**与**探索性**：
  - $\alpha$ 大 → 更注重探索（熵高）
  - $\alpha$ 小 → 更注重利用（奖励高）


> **自动调整熵正则项**：为了让 $\alpha$ 根据 **策略熵** 和 **目标熵** 的差异自己变，使实际策略熵接近 **目标熵**（自定超参数），引入一个$\alpha$ 的损失函数：
$$J(\alpha)=\mathbb{E}_{a_t\sim\pi_\phi}\left[-\alpha\left(\log\pi_\phi(a_t|s_t)+\mathcal{H}_{\mathrm{target}}\right)\right]$$
$$\frac{\partial J}{\partial\alpha}=\mathbb{E}_{a_t\sim\pi_\phi}\left[-\left(\log\pi_\phi(a_t|s_t)+\mathcal{H}_{\mathrm{target}}\right)\right]$$
- 如果实际熵>目标熵，括号里的值为负，梯度让$\alpha$变小，降低随机性
- 如果实际熵<目标熵，括号里的值为正，梯度让$\alpha$变大，增加随机性
> **自动调整熵正则项$\alpha$** 有助于加速策略学习，并兼顾全局，减少策略陷入较差的局部最优的可能性
> 经验：**目标熵数值** 一般在负的动作维度数附近

![传统强化学习和最大熵强化学习的区别](Illustrations/传统强化学习和最大熵强化学习的区别.png)

### Soft 策略迭代
> 普通的策略迭代：**策略评估（Policy Evaluation）** + **策略提升（Policy Improvement）** 

> **Soft** 版本的 **Policy Improvement** ，策略的更新由 **选择动作价值最大的动作（[动态规划中的策略提升定理](../RL_Fundamentals/5.动态规划(dynamic_programming).ipynb)）** 变为了 Boltzmann分布（**Soft**max）：
$$\pi_{new}(a|s)=\frac{e^{\frac{Q_{old}(s,a)}{\alpha}}}{\sum_{a^{\prime}}e^{\frac{Q_{old}(s,a^{\prime})}{\alpha}}}$$
- 这种方法在离散动作空间下比较容易实现，因为可以直接枚举动作求分布。但在 连续动作空间 下，计算归一化因子（积分）非常困难，近似计算代价很大。SAC 使用一个 **Actor 表示策略函数** 来解决这个问题
- 其推导的过程是根据 **目标函数$J(\pi)$** 与 **约束条件$\int_a\pi(a|s)da=1$** 用 **拉格朗日乘子法** 构造无约束优化，再对$\pi(a|s)$求导，令导数为0得出
> **Soft** 版本的 **Policy Evaluation** 中 **状态价值函数**：
$$V(s_t)=\mathbb{E}_{a_t\sim\pi}[Q(s_t,a_t)-\alpha\log\pi(a_t|s_t)]=\mathbb{E}_{a_t\sim\pi}[Q(s_t,a_t)]+H(\pi(\cdot|s_t))$$
> 所以 **Soft 贝尔曼方程**：
$$Q(s,a)=r(s,a)+\gamma\mathbb{E}_{s^{\prime}}\left[\mathbb{E}_{a^{\prime}\sim\pi}\left[Q(s^{\prime},a^{\prime})-\alpha\log\pi(a^{\prime}|s^{\prime})\right]\right]=r(s,a)+\gamma\mathbb{E}_{s^{\prime}}\left[\mathbb{E}_{a^{\prime}\sim\pi}[Q(s^{\prime},a^{\prime})]+H(\pi(\cdot|s^{\prime}))\right]$$
> 

### SAC 网络结构
> **SAC** 算法通常包含 5 个网络：
1. **两个 Q 网络 $Q_{\omega}$（Critics）**：参数分别为$\omega_1$和$\omega_2$，基于 **Double DQN** 的思想，每次会挑选Q值小的网络，从而缓解**Q值过高**估计的问题
2. **两个目标 Q 网络 $Q_{\omega^{-}}$**：对应**两个 Q 网络**，参数分别为$\omega_1^-$和$\omega_2^-$，是为了让训练**更加稳定**，更新方式与 **DDPG** 中的更新方式一样。再根据 **策略过去收集的数据*R*** 有 **Q 网络的损失函数** ：
$$\begin{aligned}
L_{Q}(\omega) & =\mathbb{E}_{(s_{t},a_{t},r_{t},s_{t+1})\sim R}\left[\frac{1}{2}\left(Q_{\omega}(s_{t},a_{t})-(r_{t}+\gamma V_{\omega^{-}}(s_{t+1}))\right)^{2}\right] \\
 & =\mathbb{E}_{(s_t,a_t,r_t,s_{t+1})\sim R,a_{t+1}\sim\pi_\theta(\cdot|s_{t+1})}\left[\frac{1}{2}\left(Q_\omega(s_t,a_t)-(r_t+\gamma(\min_{j=1,2}Q_{\omega_j^-}(s_{t+1},a_{t+1})-\alpha\log\pi(a_{t+1}|s_{t+1})))\right)\right]^2
\end{aligned}$$
3. **一个 策略网络（Actor）**：之前提到策略的更新公式：$\pi_{new}(a|s)=\frac{e^{\frac{Q_{old}(s,a)}{\alpha}}}{\sum_{a^{\prime}}e^{\frac{Q_{old}(s,a^{\prime})}{\alpha}}}$，但是其在 **连续动作空间** 下无法很好的应用。于是结合 KL 散度，通过**最小化** KL 散度：$$\pi_{\mathrm{new}}=\arg\min_{\pi^{\prime}}D_{KL}\left(\pi^{\prime}(\cdot|s),\frac{\exp(\frac{1}{\alpha}Q^{\pi_{\mathrm{old}}}(s,\cdot))}{Z^{\pi_{\mathrm{old}}}(s,\cdot)}\right)$$
- 可 **吸收掉** 归一化因子$Z(s)=\int_A\exp\left(\frac{Q(s,a^{\prime})}{\alpha}\right)da^{\prime}$
- 从而得到 **策略网络的损失函数**：$$L_\pi(\theta)=\mathbb{E}_{s_t\sim R,a_t\sim\pi_\theta}[\alpha\log(\pi_\theta(a_t|s_t))-Q_\omega(s_t,a_t)]=\mathbb{E}_{s_t\sim R,a_t\sim\pi_\theta}[\alpha\log(\pi_\theta(a_t|s_t))-\min_{j=1,2}Q_{\omega_j}(s_t,a_t)]$$
- 可以看出本质上也是 **最大化状态价值函数$V$**

##### 采样动作不可导问题
> 实际上，**$Q$函数** 当中的 **动作$a$** 与 **策略网络** 有联系，**$Q$函数** 可进行对 **策略网络参数** 的梯度求解，且让梯度穿过 **$Q$函数** 的 **动作$a$** 直接反传到 **策略网络** 使算法用的是 **确定性导数信息**，可以使 **策略网络** 更新更为稳定，降低方差：
- 之前传统 **REINFORCE** 风格的梯度，并没有对 **$Q$函数** 对进行操作，只对 **策略的对数概率密度函数** 进行梯度求解
- 而 **DDPG** 则是通过 **确定性策略** 天然避免了 **采样不可导** 的问题
> **随机性策略** 的 **SAC** 则是使用 **重参数化技巧（reparameterization trick）**，使得
$$a=f_\phi(s,\epsilon)=\mu_\phi(s)+\sigma_\phi(s)\cdot\epsilon,\quad\epsilon\sim\mathcal{N}(0,1)$$
> 以此实现 **动作可导**

> 最后导出 **损失函数**:
$$L_\pi(\theta)=\mathbb{E}_{s_t\sim R,\epsilon_t\sim\mathcal{N}}\left[\alpha\log(\pi_\theta(f_\theta(\epsilon_t;s_t)|s_t))-\min_{j=1,2}Q_{\omega_j}(s_t,f_\theta(\epsilon_t;s_t))\right]$$